<a href="https://colab.research.google.com/github/Ivan412/training/blob/main/6_python_advaced/2_import_and_parsing/task4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Продвинутый Python. Часть 2: Импорт и парсинг данных. Работа с SQL.

---


#**Задание: загрузка результатов в БД**

Соберите данные о моделях холодильников Саратов с маркетплейса beru.ru: URL, название, цена, размеры, общий объем, объем холодильной камеры.

Создайте соответствующие таблицы в SQLite базе данных и загрузите полученные данные в таблицу beru_goods.

Для парсинга можно использовать зеркало страницы beru.ru с результатами для холодильников Саратов по адресу:

https://video.ittensive.com/data/018-python-advanced/beru.ru/

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import sqlite3
import re

In [2]:
#шаблоны поиска
re_templates = {'freezer': re.compile(r'.*Холодильник\s*Саратов.*', re.IGNORECASE)}

In [3]:
def get_links(text_pattern, bs_html):
  '''
  Функция возвращает локальные ссылки на товары из родительских тэгов этих товаров
  '''
  urls = []
  #находим товары по шаблону
  tags = bs_html.find_all('span', string=text_pattern)
  #для каждого товара ищем ссылку в его рродителях
  for tag in tags:
    try:
      for parent in tag.parents:
        if (parent.name == 'a') and (parent.has_attr('href')):
          urls.append(parent['href'])
    except:
      pass
  return urls


def get_features(url):
  '''
  Функция возвращает характеристики товара:
  url, name, price, length, width, height, vol, freezer_vol
  '''
  #regexp для именованных групп вида decimal с разделителями 'x'
  lwh_pattern = re.compile('(?P<w>\d+(?:\.\d+)?)х(?P<h>\d+(?:\.\d+)?)х(?P<l>\d+(?:\.\d+)?)')
  #regexp для именованных групп объемов
  vol_pattern = re.compile('общий объем (?P<vol>\d+(?:\.\d+)?) л')
  fvol_pattern = re.compile('объем холодильной камеры (?P<fvol>\d+(?:\.\d+)?) л')

  result = []
  result.append(url)

  #откроим страницу с товаром или вернем заглушку
  r = requests.get(url)
  if r.status_code != 200:
    result.extend([''] * 7)
    return result
  bs_html = bs(r.content, 'lxml')

  #найдем название
  name = bs_html.find('h1')
  result.append(name.getText()) if name else result.append('')

  #найдем цену
  price = bs_html.find('span', attrs={'data-tid': 'c3eaad93'})
  result.append(price.getText().replace(' ', '')) if price else result.append('')

  #найдем характеристики
  specs = bs_html.find('div', attrs={'data-zone-name': 'specs'})

  #найдем размеры
  try:
    lwh = lwh_pattern.search(specs.getText()).groupdict()
    l, w, h = lwh['w'], lwh['l'], lwh['h']
    result.extend(map(float, [l, w, h]))
  except:
    result.extend([''] * 3)

  #найдем общий объем
  vol = vol_pattern.search(specs.getText())
  result.append(float(vol.groupdict()['vol'])) if vol  else result.append('')

  #найдем объем холодильной камеры
  fvol = fvol_pattern.search(specs.getText())
  result.append(float(fvol.groupdict()['fvol'])) if fvol else result.append('')
  
  return result

In [7]:
#откроем страничку с товарами
url = 'https://video.ittensive.com/data/018-python-advanced/beru.ru/'
r = requests.get(url)
html = bs(r.content, 'lxml')
#получим относительные и абсолютные ссылки на товары
links = get_links(re_templates['freezer'], html)
links = [url + x for x in links]
#получим характеристики по каждому товару
data = []
for link in links:
  line = get_features(link)
  data.append(line)
  print(f'Добавлен товар: {line[1]}')

Добавлен товар: Холодильник Саратов 452 (КШ-120)
Добавлен товар: Холодильник Саратов 263 (КШД-200/30)
Добавлен товар: Холодильник Саратов 209 (КШД 275/65)
Добавлен товар: Холодильник Саратов 209 белый с черными накладками
Добавлен товар: Холодильник Саратов 452 (КШ-120) серый
Добавлен товар: Холодильник Саратов 550 (КШ-120 без НТО)
Добавлен товар: Холодильник Саратов 467 (КШ-210)
Добавлен товар: Холодильник Саратов 264 (КШД-150/30)
Добавлен товар: Холодильник Саратов 264 (КШД-150/30) серый
Добавлен товар: Холодильник Саратов 263 серый


In [8]:
#присоеденимся к базе и создадим таблицу
conn = sqlite3.connect("data.db3")
db = conn.cursor()
db.execute("""CREATE TABLE beru_goods
            (id INTEGER PRIMARY KEY AUTOINCREMENT not null,
            url text,
            name text default '',
            price DECIMAL default 0.0,
            width FLOAT default 0.0,
            depth FLOAT default 0.0,
            height FLOAT default 0.0,
            volume FLOAT default 0.0,
            freezer FLOAT default 0.0)""")
conn.commit()

In [9]:
#вставим данные в таблицу
db.executemany("""INSERT INTO beru_goods (url, name, price, width, depth, height, volume, freezer)
           VALUES (?, ?, ?, ?, ?, ?, ?, ?)""", data)
conn.commit()

In [10]:
#посмотрим содержимое таблицы
for line in db.execute("SELECT * FROM beru_goods").fetchall():
  print(line)
  db.close()

(1, 'https://video.ittensive.com/data/018-python-advanced/beru.ru/kholodilnik-saratov-452-ksh-120.html', 'Холодильник Саратов 452 (КШ-120)', 10728, 48.0, 60.0, 89.6, 122.0, 107.0)
(2, 'https://video.ittensive.com/data/018-python-advanced/beru.ru/kholodilnik-saratov-263-kshd-200-30.html', 'Холодильник Саратов 263 (КШД-200/30)', 14299, 48.0, 60.0, 149.0, 195.0, 165.0)
(3, 'https://video.ittensive.com/data/018-python-advanced/beru.ru/kholodilnik-saratov-209-kshd-275-65.html', 'Холодильник Саратов 209 (КШД 275/65)', 13292, 60.0, 60.6, 163.0, 275.0, 210.0)
(4, 'https://video.ittensive.com/data/018-python-advanced/beru.ru/kholodilnik-saratov-209-belyi-s-chernymi-nakladkami.html', 'Холодильник Саратов 209 белый с черными накладками', 18259, 60.0, 61.0, 163.0, 275.0, 210.0)
(5, 'https://video.ittensive.com/data/018-python-advanced/beru.ru/kholodilnik-saratov-452-ksh-120-seryi.html', 'Холодильник Саратов 452 (КШ-120) серый', 10946, 48.0, 60.0, 89.6, 122.0, 107.0)
(6, 'https://video.ittensive.co